## 月に1日だけログインしている人の分析

In [183]:
from pathlib import Path
import matplotlib
import math
import re
import datetime as dt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import sys
sys.path.append('../../../utility/python/')
from utility import (
    semi_join, anti_join, preprocess_web_logs_screen_name, complement_data,
    sub_days, plus_day, add_group_num, plot_two_axis_bar_line,plus_month
)
from utility import load_web_logs_from_bigquery, load_daily_user_logs_from_bigquery, load_data_from_db_with_script

In [184]:
%load_ext autoreload
%autoreload 2
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [185]:
data_dir_path = Path('data')
result_dir_path = Path('result')
if not data_dir_path.exists():
    data_dir_path.mkdir(parents=True)

In [186]:
target_day = '2022-11-03'
target_month = target_day[:7]

In [187]:
mazrica_domain_ids = [1, 8, 2095]
privilege_replace_dict = {1: '1_管理職', 2: '2_一般', 3: '2_一般'}
daily_user_logs_name_dict = {
    'deal_board': '案件ボード',
    'deals_table': '案件一覧',
    'actions_table': 'アクション一覧',
    'customers_table': '取引先一覧',
    'contacts_table': 'コンタクト一覧',
    'campaigns_table': 'キャンペーン一覧',
    'webforms_table': 'ウェブフォーム一覧',
    'custom_reports_table': 'カスタムレポート一覧',
    'dashboards_table': 'ダッシュボード一覧',
    'deal_page': '案件ページ',
    'action_page': 'アクションページ',
    'customer_page': '取引先ページ',
    'contact_page': 'コンタクトページ',
    'campaign_page': 'キャンペーンページ',
    'webform_page': 'ウェブフォームページ',
    'custom_report_page': 'カスタムレポートページ',
    'dashboard_page': 'ダッシュボードページ',
    'notification_modal_page': '通知モーダルページ',
    'reports': 'レポート系',
    'customer_merge_candidate': '名寄せ_取引先',
    'contact_merge_candidate': '名寄せ_コンタクト'
}

### データダウンロード

In [294]:
domain_user_maps = pd.read_csv('domain_user_maps.csv')
daily_user_logs = pd.read_csv(data_dir_path.joinpath('daily_user_logs.tsv'), sep='\t')

#### 前処理
マツリカユーザー削除・契約タイプ制限・dateをdatetime型へ変換

In [295]:
dodomain_user_maps = anti_join(
    domain_user_maps, pd.DataFrame({'domain_id': mazrica_domain_ids}), by='domain_id'
).query('use_type == 30 or use_type == 40', engine='python')
daily_user_logs = daily_user_logs.assign(date=lambda x: x.date.astype(str)).sort_values(['date'])
domain_user_maps.head()

,domain_id,user_id,use_type,user_license,user_active,domain_active,privilege,contract_start_date,created_at
0,1,1,40.0,150,0,1,1,2016-01-01,2015-07-27 01:24:03
1,1,3,40.0,150,0,1,1,2016-01-01,2015-07-27 01:33:05
2,1,4,40.0,150,0,1,2,2016-01-01,2015-07-29 03:01:45
3,1,5,40.0,150,0,1,1,2016-01-01,2015-07-27 07:18:44
4,1,6,40.0,150,0,1,1,2016-01-01,2015-07-27 10:59:47


### それぞれのユーザーが毎月何回ログインしたか

In [296]:
login_count = daily_user_logs.query('login_web == 1', engine='python').assign(
    month = lambda x: x.date.map(lambda y: dt.datetime.strptime(
        y, '%Y-%m-%d'
    ).strftime('%Y-%m'))
).query(f'month <= "{plus_month(target_day, -1)}"', engine='python').groupby(
    ['month', 'user_id']
)['date'].nunique().reset_index().rename(columns={'date': 'n_login_day'}).assign(
    month = lambda x: x.month.map(lambda y: dt.datetime.strptime(y, '%Y-%m').strftime('%Y年%m月'))
).pivot_table(values = ['n_login_day'], index = ['user_id'], columns = ['month'], fill_value=0)

login_count.columns = list(map(lambda x: f'{x[1]}', login_count.columns))
login_count

,2022年03月,2022年04月,2022年05月,2022年06月,2022年07月,2022年08月,2022年09月,2022年10月
user_id,,,,,,,,
19,6,7,9,5,9,13,7,7
21,0,1,0,2,1,2,2,4
104,23,21,20,22,20,21,22,20
106,12,14,11,8,12,21,20,13
108,25,22,21,22,27,26,24,26
...,...,...,...,...,...,...,...,...
20975,0,0,0,0,0,0,0,1
20976,0,0,0,0,0,0,0,1
20982,0,0,0,0,0,0,0,1


### 8ヶ月の中で1ヶ月でも1回だけをログインしたユーザーを抽出

In [297]:
login_count1 = login_count[(login_count["2022年03月"] == 1 ) | (login_count["2022年04月"] == 1)|(login_count["2022年05月"] == 1 ) | (login_count["2022年06月"] == 1 ) | (login_count["2022年07月"] == 1 ) | (login_count["2022年08月"] == 1 ) | (login_count["2022年09月"] == 1 ) | (login_count["2022年10月"] == 1 )]
login_count1.reset_index(inplace = True, drop = False)
login_count1.head(20)

,user_id,2022年03月,2022年04月,2022年05月,2022年06月,2022年07月,2022年08月,2022年09月,2022年10月
0,21,0,1,0,2,1,2,2,4
1,173,0,0,0,1,2,1,1,3
2,174,4,4,1,5,2,1,4,5
3,184,0,3,1,1,0,4,1,7
4,594,2,0,2,0,0,1,0,0
5,692,0,1,0,0,0,0,0,0
6,900,16,12,1,1,2,2,2,0
7,1243,4,1,3,5,6,1,3,2
8,1651,1,2,3,1,3,1,1,2
9,2184,0,1,0,0,1,0,0,0


### それぞれのユーザーがログイン日数のグループに何回入ったかの集計

In [298]:
(login_count1.iloc[1, 1:9]==2).sum()

1

In [299]:
n0 = []
n1 = []
n2 = []
n3= []
n4 = []
n5 = []
n6 = []
n7 = []
n8 = []
n9 = []
n10 = []

for i in range(len(login_count1.index)):
   
   value1 = (login_count1.iloc[i, 1:9] == 1).sum()
   value2 = (login_count1.iloc[i, 1:9] == 2).sum()
   value3 = (login_count1.iloc[i, 1:9] == 3).sum()
   value4 = (login_count1.iloc[i, 1:9] == 4).sum()
   value5 = (login_count1.iloc[i, 1:9] == 5).sum()
   value6 = (login_count1.iloc[i, 1:9] == 6).sum()
   value7 = (login_count1.iloc[i, 1:9] == 7).sum()
   value8 = (login_count1.iloc[i, 1:9] == 8).sum()
   value9 = (login_count1.iloc[i, 1:9] == 9).sum()
   value10= (login_count1.iloc[i, 1:9] == 10).sum()

   #value21 = (login_count1.iloc[i, 1:9] > 20).sum()

   n1.append(value1)
   n2.append(value2)
   n3.append(value3)
   n4.append(value4)
   n5.append(value5)
   n6.append(value6)
   n7.append(value7)
   n8.append(value8)
   n9.append(value9)
   n10.append(value10)
 
   #n21.append(value21)

#print(n1)
#a1=pd.DataFrame(n1)
login_count1['1日'] = pd.DataFrame(n1)
login_count1['2日'] = pd.DataFrame(n2)
login_count1['3日'] = pd.DataFrame(n3)
login_count1['4日'] = pd.DataFrame(n4)
login_count1['5日'] = pd.DataFrame(n5)
login_count1['6日'] = pd.DataFrame(n6)
login_count1['7日'] = pd.DataFrame(n7)
login_count1['8日'] = pd.DataFrame(n8)
login_count1['9日'] = pd.DataFrame(n9)
login_count1['10日'] = pd.DataFrame(n10)
#login_count1['21日〜'] = pd.DataFrame(n21)

plot_data = login_count1.drop(['2022年03月', '2022年04月', '2022年05月', '2022年06月', '2022年07月', '2022年08月', '2022年09月', '2022年10月'], axis='columns')
plot_data

/var/folders/jx/1hv5jkh52pq96p8vxbbkxljh0000gn/T/ipykernel_52307/2591462712.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  login_count1['1日'] = pd.DataFrame(n1)
/var/folders/jx/1hv5jkh52pq96p8vxbbkxljh0000gn/T/ipykernel_52307/2591462712.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  login_count1['2日'] = pd.DataFrame(n2)
/var/folders/jx/1hv5jkh52pq96p8vxbbkxljh0000gn/T/ipykernel_52307/2591462712.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try

,user_id,1日,2日,3日,4日,5日,6日,7日,8日,9日,10日
0,21,2,3,0,1,0,0,0,0,0,0
1,173,3,1,1,0,0,0,0,0,0,0
2,174,2,1,0,3,2,0,0,0,0,0
3,184,3,0,1,1,0,0,1,0,0,0
4,594,1,2,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
2779,20975,1,0,0,0,0,0,0,0,0,0
2780,20976,1,0,0,0,0,0,0,0,0,0
2781,20982,1,0,0,0,0,0,0,0,0,0
2782,20984,1,0,0,0,0,0,0,0,0,0


In [323]:
label=[0,1,2,3,4,5,6,7]
day_group = pd.DataFrame(plot_data.groupby('10日')['user_id'].nunique()).rename(
    columns = {'user_id': '1日'}
)
day_group

,1日
10日,
0,2622
1,140
2,20
3,2
